In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
# Download daily data for SPY (S&P 500 ETF) since 2010
data = yf.download('SPY', start='2010-01-01', end='2025-04-09')

In [ ]:
data

In [ ]:
data["daily_change"] = data["Close"]['SPY'] / data['Open']['SPY']

In [ ]:
data_prev_day = data.shift(1)

In [ ]:
data["change"] = data["Close"]['SPY'] / data_prev_day["Close"]['SPY']

In [ ]:
data = data.fillna({('change', ''): 1})

In [ ]:
data["month_start"] = (pd.Series(data.index).dt.month != pd.Series(data.index).shift().dt.month).astype(int).values * 200

In [ ]:
total_invest = 1000

money = 1000
moneys = []
for x, y in zip(data["change"].values, data["month_start"].values):
    total_invest = total_invest + y
    money = money * x
    money = money + y
    moneys.append(money)

data["val"] = moneys

In [ ]:
dailys = []
daily = 10
daily_mult = 1
for x in data["daily_change"].values:
    if x >= 1:
        daily_mult += 1
        dailys.append(0)
    else:
        daily_quota = daily * daily_mult
        dailys.append(daily_quota)
        daily_mult = 1

data["daily_invest"] = dailys

In [ ]:
dailys = []
daily = 10
daily_mult = 1
for x in data["daily_change"].values:
    if x < 1:
        daily_mult += 1
        dailys.append(0)
    else:
        daily_quota = daily * daily_mult
        dailys.append(daily_quota)
        daily_mult = 1

data["daily_invest_high"] = dailys

In [ ]:
total_invest = 1000

money = 1000
moneys = []
for x, y in zip(data["change"].values, data["daily_invest"].values):
    total_invest = total_invest + y
    money = money * x
    money = money + y
    moneys.append(money)

data["val_daily"] = moneys

In [ ]:
total_invest = 1000

money = 1000
moneys = []
for x, y in zip(data["change"].values, data["daily_invest_high"].values):
    total_invest = total_invest + y
    money = money * x
    money = money + y
    moneys.append(money)

data["val_daily_high"] = moneys

In [ ]:
data["down_day"] = ((data["val_daily"] / (1000 + data["daily_invest"].cumsum())) - 1) * 100
data["up_day"] = ((data["val_daily_high"] / (1000 + data["daily_invest_high"].cumsum())) - 1) * 100
data["dca"]  = ((data["val"] / (1000 + data["month_start"].cumsum())) - 1) * 100

In [ ]:
data.tail()


In [ ]:
import plotly.express as px

label_map = {
    'dca': 'DCA',
    'down_day': 'Down Day',
    'up_day': 'Up Day'
}

fig = px.line(data, x=data.index, y=["dca", "down_day"], title="Percentage Change", template="plotly_dark", #"up_day"
              labels={"value": "Percentage Gain", "variable": "Legend"}
              )

fig.for_each_trace(
    lambda t: t.update(name=label_map[t.name], legendgroup=label_map[t.name], hovertemplate=t.hovertemplate.replace(t.name, label_map[t.name]))
)
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(data, x=data.index, y=["val", "val_daily"], title="Stock Price")
fig.show()

In [ ]:
data["month_start"].sum()

In [ ]:
data["daily_invest"].sum()